In [ ]:
%matplotlib inline
import tables
import numpy
import os
import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib

import glob
import sys
import matplotlib.gridspec as gridspec
import CalculateMeanWinds


# Helper functions

In [ ]:
def readafile(fname):
    '''
    Function written by nicolls to read
    '''
    h5file=tables.open_file(fname)
    output={}
    for group in h5file.walk_groups("/"):
        output[group._v_pathname]={}
        for array in h5file.list_nodes(group, classname = 'Array'):
            output[group._v_pathname][array.name]=array.read()
    h5file.close()

    return output

def WeightedMeanNaN(x, dx):
    xdx = x/dx
    # bevington 4.17
    weightedMean = numpy.nansum(x/(dx**2), axis=0)/numpy.nansum(1./dx**2, axis=0)
    # bevington 4.19
    weightedVar = 1./numpy.nansum((1./dx**2), axis=0)
    weightedStd = numpy.sqrt(weightedVar)
    n = [numpy.where(numpy.isfinite(xdx[:,i]))[0].shape[0] for i in range(xdx.shape[1])]
    # print n
    return weightedMean, weightedStd, n

def MakeQuiverFigures(DecimalHours, ZonalWinds,MeridWinds,radius=0.5):
    # can now validate using testmltplots.ipynb
    timeInDegrees = 180-360.*(DecimalHours/24.)
#     print 'timeInDegrees', DecimalHours,timeInDegrees
    theta = numpy.deg2rad(timeInDegrees)
    radii = numpy.ones(theta.shape[0])*radius
    Uin = ZonalWinds
    Vin = MeridWinds
    #test
    # Vin = 0.0
    # Uin = 1.0
    Vprime = -numpy.cos(theta)*Vin + numpy.sin(theta)*Uin
    Uprime = -numpy.sin(theta)*Vin + -numpy.cos(theta)*Uin




    # plt.show()
    return theta,radii, Vprime, Uprime

In [ ]:
def PlotFlowsAndWindsVYear(inDict,KeySelect,inalt, outfile):
    nrow = len(inDict)
    ncol = len(inalt)+1
    # with PdfPages(outFile) as pdf:
    totalNumPlots = nrow*ncol

    f = plt.figure(dpi=300,figsize=(7,9))
    gs = gridspec.GridSpec(nrow, ncol)
#     iplot1 = ialt # sets the altitude we are interested in
#     iplot2 = 7
    iloc = 1
    
    keyList = sorted(inDict.keys())
    
    # scaling of the vectors
    scale = 350.
    Wscale = 250.
    
    ikey = 0
    for irow in range(0,nrow,1):
        print ikey, keyList[ikey]
        mltTimeDict = inDict[keyList[ikey]][KeySelect]
        for icol in range(ncol):
            print 'irow,icol',totalNumPlots,irow,icol
            rticks = numpy.cos(numpy.deg2rad(numpy.arange(90.,40.,-10)))
            rPFISR = numpy.cos(numpy.deg2rad(68.))
            # plot the flows
            if icol == 0:
                ax = plt.subplot(gs[irow,icol],projection='polar')
                theta,radii,Vprime,Uprime = MakeQuiverFigures(mltTimeDict['DecimalHoursTimeGrid'], \
                                                          mltTimeDict['MeanZonalFlow'][1,:],\
                                                          mltTimeDict['MeanMeridFlow'][1,:], \
                                                          radius=rPFISR)

                Q = ax.quiver(theta, radii, Uprime, Vprime,color='blue', units='xy', scale=scale, pivot='tail')
#                 ax.quiver(0,0.2,100.,100., color='cyan', units='xy', scale=10.)
                
#                 theta,radii,Vprime,Uprime = MakeQuiverFigures(mltTimeDict['DecimalHoursTimeGrid'], \
#                                                           mltTimeDict['MeanZonalFlowFregion'][1,:],\
#                                                           mltTimeDict['MeanMeridFlowFregion'][1,:], \
#                                                           radius=rPFISR+0.1)
#                 ax.quiver(theta, radii, Uprime, Vprime,color='red')
                ax.set_theta_direction(-1)
                ax.set_theta_zero_location('N')
                ax.set_rlim([0,0.5])
                ax.set_rticks(rticks)
                ax.set_thetalim(-numpy.pi, numpy.pi) # had to add this line, https://stackoverflow.com/questions/21314894/setting-theta-ticks-in-matplotlib-polar-plots
                ax.set_xticks([-numpy.pi,-numpy.pi/2,0,numpy.pi/2])
                ax.set_xticklabels(['00', '18', '12', '06'])
                ax.set_ylabel('%s \n \n'%keyList[ikey][0:4], fontsize=14)
                ax.quiverkey(Q, 0.95, 0.95, 200, r'$200 \frac{m}{s}$', labelpos='E')
                ax.set_xlabel('300 km', fontsize=16)
    
            
            if icol != 0:
                ax = plt.subplot(gs[irow,icol],projection='polar')

                # for iplot in range(1,12,1):# MeanAltitude.shape[0]):
                # for irow in range(nrow):
                #     for icol in range(ncol):
                ialt = inalt[icol-1]
                print mltTimeDict['MeanAltitude'][ialt]
                
                theta,radii,Vprime,Uprime = MakeQuiverFigures(mltTimeDict['DecimalHoursTimeGrid'], \
                                                          mltTimeDict['MeanZonalWinds'][0,:,ialt],\
                                                          mltTimeDict['MeanMeridWinds'][0,:,ialt], \
                                                          radius=rPFISR)

                Q = ax.quiver(theta, radii, Uprime, Vprime,color='red', units='xy', scale=Wscale, pivot='tail')
                ax.set_xlabel('%s km'%mltTimeDict['MeanAltitude'][ialt], fontsize=14)
                print 'MaxMeridonalWind', numpy.nanmax(mltTimeDict['MeanMeridWinds'][0,:,ialt])
                ax.quiverkey(Q, 0.95, 0.95, 100, r'$100 \frac{m}{s}$', labelpos='E')
            
#             ax.quiver(theta, radii, Uprime, Vprime,color='blue')
            ax.set_theta_direction(-1)
            ax.set_theta_zero_location('N')
            ax.set_rlim([0,0.5])
            ax.set_rticks(rticks)
            ax.set_yticklabels([])
#             ax.set_rlabel_position([)
            ax.set_thetalim(-numpy.pi, numpy.pi) # had to add this line, https://stackoverflow.com/questions/21314894/setting-theta-ticks-in-matplotlib-polar-plots
#             ax.set_xticks([-numpy.pi,-numpy.pi/2,0,numpy.pi/2])
#             ax.set_xticklabels(['00', '18', '12', '06'])
            ax.set_xticks([-numpy.pi,3.*numpy.pi/4., numpy.pi/2., numpy.pi/4, 0.,\
                           -numpy.pi/4., -numpy.pi/2., -3.*numpy.pi/4. ]) # 00, 03, 06, 09, 12, 15, 18 ,21
            ax.set_xticklabels(['00','','06','','12','','18', ''])
            

                


            
        ikey=ikey+1
        

    #             ax[irow,icol].set_xlabel('Mean: %0.1f, Median: %0.1f'%(MeanX,MedianX))
                        # print numpy.nanmax(Uprime), numpy.nanmax(Vprime)
                        # if (irow == nrow-1) and (icol == ncol-1):
                        # ax[irow,icol].quiver([numpy.pi], [0.8], [100.], [0.], color='r')
                        # iplot+=1
    #             title = '%s - %s \n Red = Neutral Winds, Blue=Plasma Flow in Fregion'%(plottingOpts['yrmonStr'],plottingOpts['xlabels'])
    #             ax.set_title(title, fontsize=10)
    gs.tight_layout(f)
    f.savefig(outfile)

# Main Figure Generation

In [ ]:

# this config file is the test config file
#configfile = '/Users/srkaeppler/research/data/NSF_PFISR_Eregion_NeutralWinds/MeanProfiles/meanprofileTest.ini'

# this is the config file for the figures in the paper:
configfile = 'ConfigFileForPaper_02212019.ini'


fnames = ['/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201006_Winds.v0.4.5.2018.11.20.h5',
            '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201106_Winds.v0.4.5.2018.11.20.h5',
           '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201206_Winds.v0.4.5.2018.11.20.h5',
          '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201306_Winds.v0.4.5.2018.11.20.h5'
        ]
CalcMeanWinds = CalculateMeanWinds.CalculateMeanWinds(configfile)
TmpDict = CalcMeanWinds.main(fnames)
KeySelect = 'mlt'
# PlotWindsVYear(TmpDict,'mlt',[7,5,4],'PFISR_2012_2015_WindsV3Altitudes.10092018.png')
PlotFlowsAndWindsVYear(TmpDict,'mlt',[7,5], './2010_2013_PlasmaFlowVWinds3Altitude_MLTvMLat_02212019.pdf')

fnames = ['/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201406_Winds.v0.4.5.2018.11.20.h5',
         '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201506_Winds.v0.4.5.2018.11.20.h5',
         '/Users/srkaeppler/Dropbox/research/data/NSF_PFISR_Eregion_NeutralWinds_SharedData/v0.4.5.2018.11.20/MonthlyWinds/201606_Winds.v0.4.5.2018.11.20.h5']

CalcMeanWinds = CalculateMeanWinds.CalculateMeanWinds(configfile)
TmpDict = CalcMeanWinds.main(fnames)
KeySelect = 'mlt'
# PlotWindsVYear(TmpDict,'mlt',[7,5,4],'PFISR_2012_2015_WindsV3Altitudes.10092018.png')
PlotFlowsAndWindsVYear(TmpDict,'mlt',[7,5], './2014_2016_PlasmaFlowVWinds3Altitude_MLTvMLat_02212019.pdf')